In [ ]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob


from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
from sklearn.model_selection import TimeSeriesSplit
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from datetime import datetime, timedelta, timezone
from itertools import combinations, product
# DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV



import optuna
optuna.logging.set_verbosity(optuna.logging.CRITICAL)
import gc
import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
# PATH
DATA_PATH  = '../DATA'
TRAIN_PATH = os.path.join(DATA_PATH, 'train')
TRAIN_CSV  = os.path.join(DATA_PATH, 'train.csv')
SAMPLE_PATH = os.path.join(DATA_PATH, 'sample_submission.csv')

## Hyperparameter Setting

In [3]:
CFG = {
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'SEED':41
}

## Fixed RandomSeed

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    # torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = True


seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [5]:
df_train = pd.read_csv(TRAIN_CSV)
df_ss = pd.read_csv(SAMPLE_PATH)
df_train

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
0,20210601,A060310,3S,166690,2890,2970,2885,2920
1,20210601,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2,20210601,A006840,AK홀딩스,103691,35500,35600,34150,34400
3,20210601,A054620,APS,462544,14600,14950,13800,14950
4,20210601,A265520,AP시스템,131987,29150,29150,28800,29050
...,...,...,...,...,...,...,...,...
987995,20230530,A189980,흥국에프엔비,272284,3005,3035,2955,2980
987996,20230530,A000540,흥국화재,50218,3250,3255,3195,3215
987997,20230530,A003280,흥아해운,130664,1344,1395,1340,1370
987998,20230530,A037440,희림,141932,9170,9260,9170,9200


## columns 
1. '일자': 이 컬럼은 특정 거래가 발생한 날짜를 나타냅니다. "YYYY-MM-DD" 형식으로 표현됩니다.

2. '종목코드': 각각의 주식을 식별하는 고유한 코드입니다. 한국의 경우 종목코드는 대부분 6자리 숫자로 이루어져 있습니다. (ex - A060310)

3. '종목명': 주식의 공식 이름을 나타냅니다. 이 이름은 주로 회사의 이름을 반영하며, 시장에서 해당 주식을 찾을 때 사용됩니다.

4. '거래량': 특정 일자에 해당 주식이 거래된 총 주식 수를 나타냅니다. 거래량은 시장의 활동 수준과 관심도를 반영하는 중요한 지표입니다.

5. '시가': 주식 시장이 개장했을 때의 첫 거래 가격을 의미합니다. 이는 해당 날의 시장 흐름을 이해하는데 도움이 됩니다.

6. '고가': 특정 일자에 해당 주식이 거래된 가장 높은 가격을 나타냅니다.

7. '저가': 특정 일자에 해당 주식이 거래된 가장 낮은 가격을 나타냅니다.

8. '종가': 주식 시장이 마감했을 때의 마지막 거래 가격을 의미합니다. 종가는 해당 일의 주식 가격 변동을 반영하며, 이후의 시장 분석에 중요한 기준이 됩니다.

In [12]:
# change columns name
df = df_train.rename(columns={
    '일자': 'date',
    '종목코드': 'code',
    '종목명': 'name',
    '거래량': 'volume',
    '시가': 'open',
    '고가': 'high',
    '저가': 'low',
    '종가': 'close'
})
# datetime
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['dayofweek'] = df['date'].dt.dayofweek
df['weekofyear'] = df['date'].dt.weekofyear
df['quarter'] = df['date'].dt.quarter
df['dayofyear'] = df['date'].dt.dayofyear

df_processed = df.copy()
le = preprocessing.LabelEncoder()
df_processed['code'] = le.fit_transform(df_processed['code'])  
df_processed.drop(columns=['name'], inplace=True)

df_processed.set_index('date', inplace=True)


In [13]:
df_processed

,code,volume,open,high,low,close,year,month,day,dayofweek,weekofyear,quarter,dayofyear
date,,,,,,,,,,,,,
2021-06-01,971,166690,2890,2970,2885,2920,2021,6,1,1,22,2,152
2021-06-01,1267,63836,5860,5940,5750,5780,2021,6,1,1,22,2,152
2021-06-01,280,103691,35500,35600,34150,34400,2021,6,1,1,22,2,152
2021-06-01,929,462544,14600,14950,13800,14950,2021,6,1,1,22,2,152
2021-06-01,1789,131987,29150,29150,28800,29050,2021,6,1,1,22,2,152
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-30,1562,272284,3005,3035,2955,2980,2023,5,30,1,22,2,150
2023-05-30,27,50218,3250,3255,3195,3215,2023,5,30,1,22,2,150
2023-05-30,146,130664,1344,1395,1340,1370,2023,5,30,1,22,2,150


In [24]:
# target 값 생성
df_processed['target'] = df_processed.groupby('code')['close'].pct_change()
df_processed.fillna(0, inplace=True)
print(df_processed.isnull().sum())


code          0
volume        0
open          0
high          0
low           0
close         0
year          0
month         0
day           0
dayofweek     0
weekofyear    0
quarter       0
dayofyear     0
target        0
dtype: int64


In [25]:
pd.options.display.float_format = '{:,.6g}'.format
df_processed.describe()

,code,volume,open,high,low,close,year,month,day,dayofweek,weekofyear,quarter,dayofyear,target
count,"988,000","988,000","988,000","988,000","988,000","988,000","988,000","988,000","988,000","988,000","988,000","988,000","988,000","988,000"
mean,999.5,"773,427","25,877","26,349.4","25,423.3","26,002.8","2,021.91",6.55263,15.83,2.04049,26.5405,2.51417,183.917,8.08968e-05
std,577.35,4.83991e+06,"65,908.4","66,898.2","64,903.1","65,870.8",0.702392,3.44715,8.6803,1.39924,14.9783,1.11613,105.266,0.0582078
min,0,0,0,0,0,85,"2,021",1,1,0,1,1,2,-0.912715
25%,499.75,"36,539","3,775","3,865","3,700","3,890","2,021",4,8,1,14,2,94,-0.0144462
50%,999.5,"114,022","8,630","8,810","8,470","8,700","2,022",7,16,2,26.5,3,184,0
75%,"1,499.25","367,300","21,300","21,800","20,850","21,400","2,022",10,23,3,40,4,277,0.0112233
max,"1,999",7.82243e+08,1.77e+06,1.784e+06,1.756e+06,1.77e+06,"2,023",12,31,4,52,4,364,27.1092


In [26]:
# time series train test split
def ts_train_test_split(df, test_size=0.05):
    df_prices = df.iloc[:-int(len(df)*test_size)]
    prices = df.iloc[-int(len(df)*test_size):]
    return df_prices, prices
df_prices, prices = ts_train_test_split(df_processed)


In [27]:
prices.index

DatetimeIndex(['2023-04-21', '2023-04-21', '2023-04-21', '2023-04-21',
               '2023-04-21', '2023-04-21', '2023-04-21', '2023-04-21',
               '2023-04-21', '2023-04-21',
               ...
               '2023-05-30', '2023-05-30', '2023-05-30', '2023-05-30',
               '2023-05-30', '2023-05-30', '2023-05-30', '2023-05-30',
               '2023-05-30', '2023-05-30'],
              dtype='datetime64[ns]', name='date', length=49400, freq=None)

## Sharpe ratio 측정 함수

In [28]:
def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['rank'].min() == 0
        assert df['rank'].max() == len(df['rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='rank')['target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='rank', ascending=False)['target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio
# # 가정한 데이터프레임 생성
# data = {
#     'date': pd.date_range(start='2023-05-31', end='2023-06-24').repeat(2000),  # 10개의 주식
#     'stock': np.tile(['stock' + str(i) for i in range(2000)], 15),  # 15일
#     'target': np.random.rand(2000*15),  # 예상 수익률을 임의로 생성
# }

# df = pd.DataFrame(data)
# df['rank'] = df.groupby('date')['target'].rank(method='first') - 1  # 순위 생성

# # calc_spread_return_sharpe 함수를 사용하여 6월 1일부터 6월 15일까지 Sharpe Ratio 계산
# sharpe_ratio = calc_spread_return_sharpe(df, portfolio_size=20, toprank_weight_ratio=2)

# print(f'The Sharpe Ratio from June 1 to June 15 is {sharpe_ratio:.2f}')

# # calc_spread_return_sharpe 함수를 사용하여 6월 1일부터 6월 15일까지 Sharpe Ratio 계산
# sharpe_ratio = calc_spread_return_sharpe(df, portfolio_size=100, toprank_weight_ratio=2)

# print(f'The Sharpe Ratio from June 1 to June 15 is {sharpe_ratio:.2f}')

In [29]:
# Utilities 

def calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
    weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
    weights_mean = weights.mean()
    df = df.sort_values(by='rank')
    purchase = (df['target'][:portfolio_size]  * weights).sum() / weights_mean
    short    = (df['target'][-portfolio_size:] * weights[::-1]).sum() / weights_mean
    return purchase - short
def calc_spread_return_sharpe(df, portfolio_size=200, toprank_weight_ratio=2):
    grp = df.groupby('date')
    min_size = grp["target"].count().min()
    if min_size<2*portfolio_size:
        portfolio_size=min_size//2
        if portfolio_size<1:
            return 0, None
    buf = grp.apply(calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio, buf

def add_rank(df, col_name="pred"):
    df["rank"] = df.groupby("date")[col_name].rank(ascending=False, method="first") - 1 
    df["rank"] = df["rank"].astype("int")
    return df

In [30]:
def adjuster(df):
    def calc_pred(df, x, y, z):
        return df['target'].where(df['target'].abs() < x, df['target'] * y + np.sign(df['target']) * z)

    def objective(trial, df):
        x = trial.suggest_uniform('x', 0, 0.2)
        y = trial.suggest_uniform('y', 0, 0.05)
        z = trial.suggest_uniform('z', 0, 1e-3)
        df["rank"] = calc_pred(df, x, y, z).rank(ascending=False, method="first") - 1 
        return calc_spread_return_per_day(df, 200, 2)

    def predictor_per_day(df):
        study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=SD))#5187
        study.optimize(lambda trial: abs(objective(trial, df) - 3), 3)
        return calc_pred(df, *study.best_params.values())

    return df.groupby("date").apply(predictor_per_day).reset_index(level=0, drop=True)

def _predictor_base(feature_df):
    return model.predict(feature_df[feats])

def _predictor_with_adjuster(feature_df):
    df_pred = feature_df.copy()
    df_pred["target"] = model.predict(feature_df[feats])
    return adjuster(df_pred).values.T

In [32]:
df_prices.columns

Index(['code', 'volume', 'open', 'high', 'low', 'close', 'year', 'month',
       'day', 'dayofweek', 'weekofyear', 'quarter', 'dayofyear', 'target'],
      dtype='object')

In [34]:
# 1. 특성 엔지니어링 - 원본 데이터에 포함된 모든 특성을 포함합니다.
# 'close', 'open', 'high', 'low', 'volatility', 'daily_change' 등
# 만약 추가로 생성한 특성이 있다면, 여기에 포함해야 합니다.
df_prices['volatility'] = df_prices['high'] - df_prices['low']  # 일일 변동성
df_prices['daily_change'] = df_prices['close'] - df_prices['open']  # 일일 가격 변동

features = ['close', 'open', 'high', 'low', 'volatility', 'daily_change','year', 'month',
       'day', 'dayofweek']  # Add all your features here

# 2. 파라미터 최적화를 위한 그리드 서치 설정
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10, 15],
}

# 3. 랜덤 포레스트 모델을 설정하고 그리드 서치를 실행
model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(df_prices[features], df_prices['target'])

# 최적의 파라미터로 모델을 재학습
best_model = grid_search.best_estimator_

# 4. 예측 및 성능 평가
df_prices["pred"] = best_model.predict(df_prices[features])
score, buf = calc_spread_return_sharpe(add_rank(df_prices))

print(f'Best parameters: {grid_search.best_params_}')
print(f'Sharpe Ratio Score -> {score}')
# 코드, 평균 최종 수익률, 순위(rank)를 포함하는 데이터프레임 생성
prices = df_prices.copy()
prices.to_csv(os.path.join(RECORDER_DIR, 'prices.csv'), index=False)
df_final = prices.groupby('code')['pred'].mean().reset_index()
df_final = df_final.sort_values(by='pred', ascending=False)
print(df_final)
df_final.reset_index(drop=True, inplace=True)
df_final['rank'] = df_final.index+1
df_final['code']= le.inverse_transform(df_final['code'].astype(int))
df_final = df_final[['code', 'rank']]
df_final.rename(columns={'code': '종목코드', 'rank': '순위'}, inplace=True)
df_final


In [ ]:
# 특성 중요도를 계산합니다.
importances = best_model.feature_importances_

# 특성 중요도를 내림차순으로 정렬합니다.
indices = np.argsort(importances)[::-1]

# 특성 중요도를 그래프로 표현합니다.
plt.figure(figsize=(12,6))
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", align="center")
plt.xticks(range(X.shape[1]), X.columns[indices], rotation=90)
plt.xlim([-1, X.shape[1]])
plt.show()

In [23]:
# DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from tqdm.auto import tqdm

# 1. 특성 엔지니어링
# 'close' 값만을 사용하는 것이 아니라 'open', 'high', 'low' 값을 이용해 추가 특성을 만듭니다.
df_prices['volatility'] = df_prices['high'] - df_prices['low']  # 일일 변동성
df_prices['daily_change'] = df_prices['close'] - df_prices['open']  # 일일 가격 변동

# 2. 파라미터 최적화를 위한 그리드 서치 설정
param_grid = {
    'n_estimators': [50, 100, 200,300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10, 15],
}

# 3. 랜덤 포레스트 모델을 설정하고 그리드 서치를 실행
model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(df_prices[['close', 'volatility', 'daily_change']], df_prices['target'])

# 최적의 파라미터로 모델을 재학습
best_model = grid_search.best_estimator_

# 4. 예측 및 성능 평가
df_prices["pred"] = best_model.predict(df_prices[['close', 'volatility', 'daily_change']])
score, buf = calc_spread_return_sharpe(add_rank(df_prices))

print(f'Best parameters: {grid_search.best_params_}')
print(f'Sharpe Ratio Score -> {score}')


KeyError: 'target'

In [ ]:
df_prices

In [96]:
# 시간 고유값 
kst = timezone(timedelta(hours=9))        
train_serial = datetime.now(tz=kst).strftime("%Y%m%d_%H%M%S")

# 기록 경로
RECORDER_DIR = os.path.join(DATA_PATH, 'results', train_serial)

# 현재 시간 기준 폴더 생성
os.makedirs(RECORDER_DIR, exist_ok=True)    
RESULT_PATH = os.path.join(RECORDER_DIR, 'submission.csv')

# 코드, 평균 최종 수익률, 순위(rank)를 포함하는 데이터프레임 생성
prices = df_prices.copy()
prices.to_csv(os.path.join(RECORDER_DIR, 'prices.csv'), index=False)
df_final = prices.groupby('code')['pred'].mean().reset_index()
df_final = df_final.sort_values(by='pred', ascending=False)
print(df_final)
df_final.reset_index(drop=True, inplace=True)
df_final['rank'] = df_final.index+1
df_final['code']= le.inverse_transform(df_final['code'].astype(int))
df_final = df_final[['code', 'rank']]
df_final.rename(columns={'code': '종목코드', 'rank': '순위'}, inplace=True)
df_final.to_csv(RESULT_PATH, index=False)
df_final

      code        pred
1774  1774   0.0358695
997    997   0.0230417
1485  1485   0.0156984
1317  1317   0.0122419
688    688  0.00920097
...    ...         ...
1263  1263 -0.00351845
1959  1959 -0.00380476
1430  1430 -0.00397803
1039  1039 -0.00418634
1531  1531 -0.00624758

[2000 rows x 2 columns]


,종목코드,순위
0,A263540,1
1,A064520,2
2,A145210,3
3,A101670,4
4,A035460,5
...,...,...
1995,A095190,1996
1996,A347700,1997
1997,A128820,1998
1998,A067370,1999
